In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [3]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [4]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [5]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [6]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [7]:
driver = webdriver.Chrome(chromedriver)

In [8]:
# Tip of the hat to Rita for helping with these two functions !
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [9]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df


In [10]:
#run this and then close the cookies pop-up!
for i in date_list:
    
    if i == '2021-12-24':
        continue
    if i == '2021-11-25':
        continue
    if i == '2022-02-18':
        continue
    if i == '2022-02-19':
        continue
    if i == '2022-02-20':
        continue
    if i == '2022-02-21':
        continue
    if i == '2022-02-22':
        continue
    if i == '2022-02-23':
        continue
    get_site_rb(i)

In [11]:
driver.close()

In [12]:
final_df

,0,1,2,3,4,5,6,7,8,9
0,1 D. Damian Jones,SAC - C,@ HOU,Points,14.50 (-120)5.57Diff: -8.93 Line: 14.50...,14.5,-120 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
1,2 D. Davion Mitchell,SAC - G,@ HOU,Points,20.50 (+110)14.86Diff: -5.64 Line: 20.5...,20.5,+110 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
2,3 D. Dejounte Murray,SAS - G,vs. POR,Points,24.50 (-110)18.95Diff: -5.55 Line: 24.5...,24.5,-110 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
3,4 J. Jaylen Brown,BOS - G,vs. IND,Points,25.50 (-115)21.95Diff: -3.55 Line: 25.5...,25.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
4,5 D. D'Angelo Russell,MIN - G,@ DEN,Points,16.50 (-115)19.94Diff: 3.44 Line: 16.50...,16.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
...,...,...,...,...,...,...,...,...,...,...
21,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
22,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
23,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
24,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...


In [13]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [14]:
final_df.reset_index(drop=True, inplace=True)

In [15]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,1 D. Damian Jones,SAC - C,@ HOU,Points,14.50 (-120)5.57Diff: -8.93 Line: 14.50...,14.5,-120 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
1,2 D. Davion Mitchell,SAC - G,@ HOU,Points,20.50 (+110)14.86Diff: -5.64 Line: 20.5...,20.5,+110 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
2,3 D. Dejounte Murray,SAS - G,vs. POR,Points,24.50 (-110)18.95Diff: -5.55 Line: 24.5...,24.5,-110 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
3,4 J. Jaylen Brown,BOS - G,vs. IND,Points,25.50 (-115)21.95Diff: -3.55 Line: 25.5...,25.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
4,5 D. D'Angelo Russell,MIN - G,@ DEN,Points,16.50 (-115)19.94Diff: 3.44 Line: 16.50...,16.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14507,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14508,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14509,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14510,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...


In [16]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [17]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [18]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [19]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy,Position,Home/Away
0,Damian Jones,SAC,HOU,Points,14.50 (-120)5.57Diff: -8.93 Line: 14.50...,14.5,-120 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,C,@
1,Davion Mitchell,SAC,HOU,Points,20.50 (+110)14.86Diff: -5.64 Line: 20.5...,20.5,+110 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,@
2,Dejounte Murray,SAS,POR,Points,24.50 (-110)18.95Diff: -5.55 Line: 24.5...,24.5,-110 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,vs.
3,Jaylen Brown,BOS,IND,Points,25.50 (-115)21.95Diff: -3.55 Line: 25.5...,25.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,vs.
4,D'Angelo Russell,MIN,DEN,Points,16.50 (-115)19.94Diff: 3.44 Line: 16.50...,16.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,@
...,...,...,...,...,...,...,...,...,...,...,...,...
14507,Grayson Allen,MIL,BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.
14508,Nemanja Bjelica,GSW,LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,F,@
14509,DeAndre Jordan,LAL,GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,C,vs.
14510,Pat Connaughton,MIL,BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.


In [20]:
bet_info = final_df.Line.str.split(" ", expand = True)
bet_info

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,,,,,14.50,(-120)5.57Diff:,-8.93,Line:,14.50,Cost:,-120,Proj.:,5.57,u
1,,,,,,,,20.50,(+110)14.86Diff:,-5.64,Line:,20.50,Cost:,+110,Proj.:,14.86,u
2,,,,,,,,24.50,(-110)18.95Diff:,-5.55,Line:,24.50,Cost:,-110,Proj.:,18.95,u
3,,,,,,,,25.50,(-115)21.95Diff:,-3.55,Line:,25.50,Cost:,-115,Proj.:,21.95,u
4,,,,,,,,16.50,(-115)19.94Diff:,3.44,Line:,16.50,Cost:,-115,Proj.:,19.94o,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,,,,,,,,10.50,(-113)11.23Diff:,0.73,Line:,10.50,Cost:,-113,Proj.:,11.23o,None
14508,,,,,,,,6.50,(-120)6.87Diff:,0.37,Line:,6.50,Cost:,-120,Proj.:,6.87o,None
14509,,,,,,,,4.50,(-130)4.82Diff:,0.32,Line:,4.50,Cost:,-130,Proj.:,4.82o,None
14510,,,,,,,,7.50,(-111)7.19Diff:,-0.31,Line:,7.50,Cost:,-111,Proj.:,7.19,u


In [22]:
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy,Position,Home/Away,Projection,Diff,over_under
0,Damian Jones,SAC,HOU,Points,14.50,14.5,-120,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,C,@,5.57,-8.93,u
1,Davion Mitchell,SAC,HOU,Points,20.50,20.5,+110,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,@,14.86,-5.64,u
2,Dejounte Murray,SAS,POR,Points,24.50,24.5,-110,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,vs.,18.95,-5.55,u
3,Jaylen Brown,BOS,IND,Points,25.50,25.5,-115,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,vs.,21.95,-3.55,u
4,D'Angelo Russell,MIN,DEN,Points,16.50,16.5,-115,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...,G,@,19.94o,3.44,o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,Grayson Allen,MIL,BKN,Points,10.50,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.,11.23o,0.73,o
14508,Nemanja Bjelica,GSW,LAL,Points,6.50,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,F,@,6.87o,0.37,o
14509,DeAndre Jordan,LAL,GSW,Points,4.50,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,C,vs.,4.82o,0.32,o
14510,Pat Connaughton,MIL,BKN,Points,7.50,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.,7.19,-0.31,u


In [23]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [24]:
final_df = final_df.copy()

In [25]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [26]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [27]:
final_df['seven_day_accuracy'] = accuracies[8]

In [28]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [29]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [30]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [31]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [32]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [33]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [34]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [35]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [36]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [37]:
sorted_df = sorted_df.replace('TBD', None)

In [38]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [40]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x));sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11 (under),o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5 (under),o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14 (over),u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12 (under),o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14 (over),o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16 (over),o,0.46
14508,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13 (over),u,0.50
14509,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11 (over),u,0.46
14510,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0 (under),u,0.46


In [ ]:
results = sorted_df.Result.str.split(" ", expand = True)

In [44]:
sorted_df.Result = results[3];sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11,o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5,o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14,u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12,o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14,o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16,o,0.46
14508,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13,u,0.50
14509,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11,u,0.46
14510,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0,u,0.46


In [50]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46
14508,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50
14509,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46
14510,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46


In [57]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [59]:
sorted_df[sorted_df.Date=='2022-03-31']

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
50,2022-03-31,Bogdan,ATL,G,CLE,vs.,16.5,-115,14.80,-1.70,16.5,NaN,u,0.46,nan
351,2022-03-31,Alex Caruso,CHI,G,LAC,vs.,7.5,-125,8.73,1.23,7.5,NaN,o,0.46,nan
485,2022-03-31,Andre Drummond,BKN,C,MIL,vs.,10.5,-115,11.20,0.70,10.5,NaN,o,0.46,nan
739,2022-03-31,Austin Reaves,LAL,G,UTH,@,7.5,-110,8.80,1.30,7.5,NaN,o,0.46,nan
832,2022-03-31,Ayo Dosunmu,CHI,G,LAC,vs.,9.5,105,9.80,0.30,9.5,NaN,o,0.46,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13841,2022-03-31,Trae Young,ATL,G,CLE,vs.,29.5,-108,27.00,-2.50,29.5,NaN,u,0.46,nan
13971,2022-03-31,Tristan Thompson,CHI,C,LAC,vs.,4.5,-120,5.19,0.69,4.5,NaN,o,0.46,nan
14180,2022-03-31,Tyrese Maxey,PHI,G,DET,@,16.5,-113,18.80,2.30,16.5,NaN,o,0.46,nan
14314,2022-03-31,Wesley Matthews,MIL,G,BKN,@,5.5,-115,5.10,-0.40,5.5,NaN,u,0.46,nan


In [60]:
sorted_df.to_csv('betting_data.csv', index=False)

In [61]:
df = pd.read_csv('betting_data.csv');df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47,0.0
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45,0.0
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45,1.0
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45,0.0
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46,1.0
14508,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50,0.0
14509,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46,0.0
14510,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46,1.0


In [62]:
client = MongoClient()

In [63]:
client.list_database_names()

['admin', 'config', 'local', 'nba_db']

In [64]:
db = client['nba_db']

In [65]:
db.drop_collection('betting_data')

{'nIndexesWas': 1, 'ns': 'nba_db.betting_data', 'ok': 1.0}

In [66]:
db.list_collection_names()

[]

In [67]:
collection = db['betting_data']

In [68]:
collection.insert_many(df.to_dict('records'))

In [73]:
db.list_collection_names()

['betting_data']

In [76]:
cursor = db.betting_data.find()
list(cursor.limit(1))

[{'_id': ObjectId('6246960a4fba57b3e39e9976'),
  'Date': '2021-10-21',
  'Name': ' Bogdan',
  'Team': 'ATL',
  'Position': 'G',
  'Opp': 'DAL',
  'Home/Away': 'vs.',
  'Line': 15.5,
  'Moneyline': -115,
  'Projection': 16.57,
  'Diff': 1.07,
  'Pick': 15.5,
  'Result': 11.0,
  'over_under': 'o',
  'seven_day_accuracy': 0.47,
  'successful_pick': 0.0}]